# **Workshop: Introduksjon til dataflyt og transformasjon**

**Du vil lære:**
- Helt overordnet hva dataflyt og transformasjoner er, hva det innebærer og hvordan det utføres i praksis
- Litt om Google Cloud Storage og Google BigQuery
- Å laste inn, hente ut og jobbe med data fra Google BigQuery
- Gjøre enkle transformasjoner ved hjelp av datamanipuleringsverktøy
- Lage et nytt og rikere datasett med data fra flere kilder

💡 Oppgave 1-4 er utforskende i GCP og kan timeboxes til ca. 10 minutter.


# **Relevante GCP-komponenter for workshopen**
## Google Cloud Storage

Google Cloud Storage (GCS) er et filområde hvor vi kan lagre filer på ulike formater, med både strukturerte og ustrukturerte data. GCS er et fint landingspunkt for data, slik at du kan jobbe uavhengig av systemene dataene stammer fra (f.eks eksterne API-er).

Vi bruker ofte begrepet "storage bucket" for en logisk oppdeling av et filområde, tilsvarende som en filmappe på din maskin.

Data i en storage bucket er som regel ikke klargjort for analyseformål. Vi ønsker derfor å prosessere og flytte den til et annet verktøy. Et slikt verktøy kan være Google BigQuery, som er en database tilpasset analyse. Dette kan videre kobles opp mot visualiserings- og modelleringsverktøy som Colab-notebooks.

## Google BigQuery

BigQuery er en SQL-basert database som er optimalisert for analyse. I motsetning til tradisjonelle SQL-databaser er BigQuery kolonnebasert istedenfor radbasert. Dette gjør den optimalisert til å regne ut aggregerte tall. BigQuery takler tabeller med et høyt antall kolonner svært godt.


# **Bolk 1: Bli kjent med datasettene**




## **Oppgave 1: Bli kjent med Google Cloud Storage** ☁️

Gå til [Google Cloud Console](https://console.cloud.google.com), logg inn med e-posten din og velg prosjekt "data-intro" oppe i venstre hjørne.

I Google Cloud Console (GUI) for prosjektet (data-intro), finn Google Cloud Storage. Du kan enten finne GCS via menyen eller søke etter den i søkefeltet.

I prosjektet finner du en bucket med to ulike datasett. Vi skal inspisere metadataen til disse filene.

a) Hvilken filtype er de?
  
  
b) Hvor store er filene?
  

In [1]:
# TODO
# Svar a: csv
# Svar b: 1GB

## **Oppgave 2: Importer dataen til BigQuery** 🔐

For å se nærmere på innholdet i datasettene ønsker vi å flytte de til BigQuery. Gjør følgende i [Google Cloud Console](https://console.cloud.google.com):

1. Finn BigQuery i menyen
2. Velg BigQuery-prosjektet "Data intro" og deretter marker datasettet bysykkel_main
3. I menylinjen oppe til høyre, velg "Create table".
4. Under "Source" kan du velge datakilden din. Vi ønsker å velge bysykkeldatasettet fra Storage Bucket. Filformatet fant du i oppgave 1.
5. Under "Destination" kan du kalle den nye tabellen din `bysykkel_(gruppenavn)`.
6. La BigQuery definere skjema for deg, og behold ellers standard-innstillingene.
7. Trykk på "Create Table"

Nå starter en jobb med å laste inn datasettet fra GCS til BigQuery og vil ta ca. 30 sekunder.

## **Oppgave 3: Bli kjent med bysykkel-datasettet** 🚲

Når vi markerer tabellen i BigQuery som vi laget i forrige oppgave ser vi en rekke metadata, samt en preview-funksjon for å undersøke radene i datasettet vårt.

Hva finner du ut om skjema (datatypene) og innholdet?




## **Oppgave 4: Bli kjent med værdatasettet** 🌦
Vi har allerede lastet inn værdatasettet inn i BigQuery (`værdata_oslo`).

Hva finner du ut om skjema (datatypene) og innholdet for dette datasettet?



# **Bolk 2: Vasking av data**

Nå som vi har flyttet datasettene til et passende sted og blitt litt kjent med tabellenes skjema skal vi prøve å slå disse sammen!

**Autentisering**

Det første vi må gjøre er å autentisere Colab-notebooken mot BigQuery. Koden under vil generere et popup-vindu der du må godkjenne dette.



In [ ]:
# Authenticate your Google Account
# Doing so means you have access to various
# resources connected to your account, such
# as BigQuery tables, Storage buckets etc.
from google.colab import auth
auth.authenticate_user()

**BigQuery-klienten**

Nå som vi er autentisert, kan vi bruke BigQuery-biblioteket. Kodesnutten under kobler oss til BigQuery-klienten og tar samtidig i bruk `magics`-operatoren som simpelthen lar oss hente data fra en tabell.

Kjør kodesnutten under.


In [ ]:
%load_ext google.cloud.bigquery
from google.cloud.bigquery import magics
magics.context.project = 'data-intro'

**Pandas DataFrames**

Operatoren over skriver resultatet fra BigQuery ut som en _DataFrame_ fra Pandas-biblioteket. DataFrames er svært nyttige når vi jobber med tabulær data, både for datautforskning og -manipulasjon.

Vi kaller dataframesene hhv. `df_bysykkel` og `df_weather`. Kjør kodeblokkene under (husk å bytte tabell-referanse):

In [ ]:
%%bigquery df_bysykkel
SELECT * FROM `data-intro.bysykkel_main.bysykkel_august` # Bytt ut med ditt eget tabell-navn


In [ ]:
%%bigquery df_weather
SELECT * FROM `data-intro.bysykkel_main.værdata_oslo`

## **Oppgave 5: Lage et utvidet datasett** 👩🏻‍💻
Vi ønsker å slå sammen de to datasettene slik at vi kan gjøre analyse på tvers av disse. Før vi gjør det må vi finne en felles kolonne for begge datasett.

På en DataFrame-instans, `df`, har vi en rekke nyttige metode og felter, f.eks:

```python
df.shape                  # Dimensjon
df.info()                 # Oppsummering av størrelse og innhold
df.describe()             # Grunnleggende statistiske egenskaper
df.head()                 # Lister de første radene i datasettet
df.tail()                 # Lister de siste radene i datasettet
df["kolonnenavn"]         # Aksessering av spesifikk kolonne
df["kolonnenavn"].iloc[n] # Aksessere rad n i spesifikk kolonne
```

Vi må finne en kolonne med fellesdata for å kunne slå sammen tabellene. Bruk en (eller flere) av metodene over for å inspisere innholdet i de to tabellene.  

Så du noen fellesnevnere da du undersøkte innholdet i tabellene?

In [ ]:
# DITT SVAR HER
df_bysykkel['started_at']
df_weather['date']

Uten å røpe for mye er det flere gode kolonne-kandidater i `df_bysykkel` som kan brukes på én kolonne i `df_weather`. Likevel er det ikke mulig å bruke noen av de rett ut av boksen. Klarer du se hvorfor?


In [2]:
# DITT SVAR HER
# Ulikt format på dato, df_bysykkel er df_weather

## **Oppgave 6: Rydde opp i datasettene** 🧹
Transformasjoner er en stor og viktig prosess når vi jobber med data. Ofte er datasettene vi har til rådighet ikke på det formatet vi ønsker å ha de på. Å transformere data betyr å gjøre endringer, for eksempel:

- slå sammen datasett
- endre på datatyper
- fjerne duplikater
- gjøre utregninger med basis i andre kolonner
- fjerne potensielle "outliers" som kan ødelegge grunnlaget vårt for analyse

> Prosessen over kalles ofte for å "vaske" data 🧼

Vi må få dato-kolonnene til å være på samme format. En måte vi kan gjøre dette på er å fjerne klokkeslettet og kun bruke dato-delen av `started_at`. Ulempen med dette er at vi da mister informasjon vi kanskje ønsker å bruke videre i analysedelen.

Vi løser dette problemet med å lage en hjelpekolonne, altså en ny midlertidg kolonne som kun brukes når vi slår sammen datasettene




### **Oppgave 6.1**

Lag en ny kolonne i `df_bysykkel`, `trip_date`, som kun inneholder datoen fra kolonnen `started_at`. Det kan ta litt tid å oppdatere de 5.7 millioner radene vi har i tabellen.

> **Hint**: Pandas har en funksjon [`to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) som lar deg tilpasse tidspunkter, samt en funksjon [`dt.strftime`](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.strftime.html) som lar deg formatere dato til et ønsket format 🐼

In [ ]:
import pandas as pd
# DIN KODE HER
df_bysykkel['trip_date'] = pd.to_datetime(df_bysykkel['started_at']).dt.strftime['%Y-%m-%d']
df_bysykkel.head()
print(df_bysykkel.head())

In [ ]:
#@title Løsningsforslag
import pandas as pd
df_bysykkel["trip_date"] = pd.to_datetime(df_bysykkel["started_at"]).dt.strftime("%Y-%m-%d")
df_bysykkel.head()

### **Oppgave 6.2**
Forsøk å slå sammen de to datasettene med hjelpekolonnen `trip_date` ved å bruke Pandas' [merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)-funksjon. Fungerer det å slå sammen nå? Hvordan ser datasettet ut?

> **Hint:** Du kan sjekke hvilke datatyper dataframen din inneholder ved å bruke Pandas sitt `dtypes`-attributt, f.eks `df_bysykkel.dtypes`

In [4]:
# DIN KODE HER
df_merged = df_bysykkel.merge(df_weather, how='outer', left_on='trip_date', right_on='date')

NameError: ignored

In [5]:
#@title Løsningsforslag
"""
Det vil i prinsippet fungere, men det er likevel ikke riktig fordi kolonnene har
ulike datatyper - started_at er av typen datetime64 og trip_date er av typen object
"""

'\nDet vil i prinsippet fungere, men det er likevel ikke riktig fordi kolonnene har\nulike datatyper - started_at er av typen datetime64 og trip_date er av typen object\n'

### **Oppgave 6.3**
Gjør nødvendige endringer i kolonnen `trip_date` i bysykkeldatasettet og `date` i værdatasettet for å kunne slå sammen.


In [ ]:
# DIN KODE HER
import pandas as pd

df_bysykkel['trip_date'] = pd.to_datetime(df_bysykkel['trip_date'])
df_weather['date'] = pd.to_datetime(weather['date'])

In [ ]:
#@title Løsningsforslag

# Vi setter begge kolonnen til å være av type datetime (datotid).

df_bysykkel["trip_date"] = pd.to_datetime(df_bysykkel["trip_date"])
df_weather["date"] = pd.to_datetime(df_weather["date"])

### **Oppgave 6.4**
Nå skal det fungere å slå sammen datasettene! 🎉 Vi ønsker å supplere hver rad av bysykkeldatasettet med værdata fra den aktuelle dagen. Skriv kode under som oppnår dette.

> 💡 Rekkefølgen og type join (left, right, outer etc.) har noe å si når datasettene slås sammen. Tommelfingerregelen er at man alltid skal starte med den største tabellen ved en merge av ytelsesgrunner. Forskjellen på type joins kan man lese mer om [her](https://stackoverflow.com/questions/5706437/whats-the-difference-between-inner-join-left-join-right-join-and-full-join).




In [ ]:
# DIN KODE HER


In [ ]:
#@title Løsningsforslag
df_merged = df_bysykkel.merge(df_weather, left_on="trip_date", right_on="date", how="left")
df_merged.head()

### **Oppgave 6.5**
Om vi sjekker typene til kolonnene med værdata (ved bruk av `df_merged.dtypes` igjen) ser vi at verdiene i disse kolonnene er strenger. Det gjør det ikke mulig for oss å aggregere verdiene, så vi er nødt til å få de over på tallformat.

Kodesnutten under forsøker å gjøre om verdiene på temperatur og nedbørsmengde til typen `float`. Hvorfor fungerer det ikke å gjøre om kolonnen direkte?

In [ ]:
df_merged["mean_temperature"] = df_merged["mean_temperature"].astype('float')
df_merged["precipitation_amount"] = df_merged["precipitation_amount"].astype('float')

In [ ]:
#@title Løsningsforslag
"""
Kolonnen ser tilsynelatende ut til å kun bestå av tall. Hvis vi derimot inspiserer
verdiene nærmere, ser vi at noen ganger forekommer strengen "NULL" som ikke er et tall.
Pandas får derfor ikke til å gjøre om datatypen før vi gjør noe med dette.
"""

### **Oppgave 6.6**
Endre datatypen på nedbørskolonnen og temperaturkolonnen slik at det blir desimaltall.

> **Hint**: Vi er nødt til å fjerne `NULL`-verdiene vi oppdaget i forrige oppgave. Bruk `replace`-funksjonen til dette ved å sette en tom verdi, som i Python-verdenen heter `None`. Kan du tenke deg hvorfor vi bruker en tom verdi istedenfor eksempelvis 0 her?

In [ ]:
# DIN KODE HER

In [ ]:
#@title Løsningsforslag
df_merged["mean_temperature"] = df_merged["mean_temperature"].replace({'NULL': None}).astype('float')
df_merged["precipitation_amount"] = df_merged["precipitation_amount"].replace({'NULL': None}).astype('float')

"""
Vi kunne teknisk sett gjort om NULL til 0, men dette ville fått betydning for
statistiske verdier som gjennomsnitt og lignende. Hvordan man håndterer NULL-verdier
avhenger av bruksområdet og krever ofte dypere forståelse av domenet man jobber med.
"""

### **Oppgave 6.7**

Rydd opp ved å fjerne hjelpekolonnen fra `df_merged`

> **Hint**: `drop`-funksjonen kan komme godt med her 🗑


In [ ]:
# DIN KODE HER

In [ ]:
#@title Løsningsforslag
df_merged = df_merged.drop(columns="trip_date")
df_merged.head()



Vi har nå et utvidet datasett! 🎉🥂🎊


Vanligvis ville vi ha skrevet datasettet tilbake til BigQuery, men det gjør vi ikke i denne workshopen (det tar lang tid, vi har mye data😴)

Pandas har en ganske snedig funksjon som kan gjøre dette!

```df_merged.to_gbq("bysykkel_main.bysykkel_med_værdata", project_id="data-intro")```

Når vi skriver til BigQuery, vil datatypene i dataframen følge med og sette riktig skjema i BigQuery
